# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [30]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/manoloserrano/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [35]:
# Read your api key environment variable
load_dotenv(find_dotenv())
NEWS_API_KEY= os.environ.get('NEWS_API_KEY')



In [36]:
# Create a newsapi client
from newsapi import NewsApiClient

api=NewsApiClient(api_key=NEWS_API_KEY)



In [37]:
# Fetch the Bitcoin news articles
Bitcoin_Articles=api.get_everything(q='bitcoin')
print(f"Total articles about Bitcoin:{Bitcoin_Articles['totalResults']}")


Total articles about Bitcoin:9813


In [38]:
# Fetch the Ethereum news articles
Ethereum_Articles=api.get_everything(q='ethereum')
print(f"Total articles about Ethereum:{Ethereum_Articles['totalResults']}")

Total articles about Ethereum:3885


In [39]:
# Create the Bitcoin sentiment scores DataFrame
Bitcoin_df = pd.DataFrame.from_dict(Bitcoin_Articles["articles"])
Bitcoin_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'engadget', 'name': 'Engadget'}",Jon Fingas,AMC theaters will accept cryptocurrencies beyo...,You won't have to stick to Bitcoin if you're d...,https://www.engadget.com/amc-theaters-accept-e...,https://s.yimg.com/os/creatr-uploaded-images/2...,2021-09-16T13:26:42Z,You won't have to stick to Bitcoin if you're d...
1,"{'id': 'engadget', 'name': 'Engadget'}",Karissa Bell,Twitter will let users send and receive Bitcoi...,Four months after Twitter first introduced in-...,https://www.engadget.com/twitter-opens-tipping...,https://s.yimg.com/os/creatr-uploaded-images/2...,2021-09-23T17:00:17Z,Four months after Twitter first introduced in-...
2,"{'id': None, 'name': 'Blogspot.com'}",noreply@blogger.com (Unknown),Bitcoin hits strongest level since May,"<ul><li>Bitcoin, in terms of market value, ros...",https://techncruncher.blogspot.com/2021/10/bit...,https://blogger.googleusercontent.com/img/a/AV...,2021-10-06T17:17:00Z,"<ul><li>Bitcoin, in terms of market value, ros..."
3,"{'id': 'engadget', 'name': 'Engadget'}",Andrew Tarantola,Hitting the Books: How Bitcoin is somehow wort...,Bitcoin and similar blockchain-based cryptos e...,https://www.engadget.com/hitting-the-books-the...,https://s.yimg.com/os/creatr-uploaded-images/2...,2021-09-25T15:30:24Z,Bitcoin and similar blockchain-based cryptos e...
4,"{'id': None, 'name': 'Blogspot.com'}",noreply@blogger.com (Unknown),JPMorgan CEO Says Bitcoin Has No Intrinsic Val...,JPMorgan CEO Jamie Dimon is still not a Bitcoi...,https://techncruncher.blogspot.com/2021/10/jpm...,None,2021-10-04T19:49:00Z,JPMorgan CEO Jamie Dimon is still not a Bitcoi...


In [40]:
# Create the Ethereum sentiment scores DataFrame
Ethereum_df = pd.DataFrame.from_dict(Ethereum_Articles["articles"])
Ethereum_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'the-verge', 'name': 'The Verge'}",Kim Lyons,China’s central bank bans cryptocurrency trans...,China’s central bank on Friday said cryptocurr...,https://www.theverge.com/2021/9/24/22691472/ch...,https://cdn.vox-cdn.com/thumbor/mde_l3lUC4muDP...,2021-09-24T16:22:55Z,Its the countrys latest crackdown on digital c...
1,"{'id': 'engadget', 'name': 'Engadget'}",Jon Fingas,AMC theaters will accept cryptocurrencies beyo...,You won't have to stick to Bitcoin if you're d...,https://www.engadget.com/amc-theaters-accept-e...,https://s.yimg.com/os/creatr-uploaded-images/2...,2021-09-16T13:26:42Z,You won't have to stick to Bitcoin if you're d...
2,"{'id': 'engadget', 'name': 'Engadget'}",Jon Fingas,OpenSea product lead resigns over use of insid...,OpenSea isn't wasting much time after its head...,https://www.engadget.com/opensea-nft-insider-t...,https://s.yimg.com/os/creatr-uploaded-images/2...,2021-09-16T19:05:02Z,OpenSea isn't wasting much time after its head...
3,"{'id': 'engadget', 'name': 'Engadget'}",Igor Bonifacic,Robinhood will start testing crypto wallets ne...,Robinhood plans to start a cryptocurrency wall...,https://www.engadget.com/robinhood-cryptocurre...,https://s.yimg.com/os/creatr-uploaded-images/2...,2021-09-22T19:00:30Z,Robinhood plans to start a cryptocurrency wall...
4,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Music company buys NFT of 'Side-eyeing Chloe' ...,The image of two-year-old Chloe Clem making a ...,https://www.bbc.co.uk/news/world-us-canada-586...,https://ichef.bbci.co.uk/news/1024/branded_new...,2021-09-25T04:30:25Z,"image source, foundation.app/@SideEyeingChloe\..."


In [41]:
# Describe the Bitcoin Sentiment
Bitcoin_df.describe()

,source,author,title,description,url,urlToImage,publishedAt,content
count,20,20,19,20,20,17,20,20
unique,7,11,19,20,20,17,20,20
top,"{'id': None, 'name': 'Blogspot.com'}",noreply@blogger.com (Unknown),Bitcoin Bros and Nuclear Bros Have Found Commo...,Cryptocurrency mining is a wildly energy-inten...,https://gizmodo.com/mining-bitcoin-using-nucle...,https://ichef.bbci.co.uk/news/1024/branded_new...,2021-10-08T13:19:51Z,PayPal has two decades of experience in online...
freq,10,10,1,1,1,1,1,1


In [42]:
# Describe the Ethereum Sentiment
Ethereum_df.describe()

,source,author,title,description,url,urlToImage,publishedAt,content
count,20,17,20,20,20,20,20,20
unique,10,14,20,20,20,20,20,20
top,"{'id': 'business-insider', 'name': 'Business I...",ilee@insider.com (Isabelle Lee),"Bitcoin rises 9.5% to $47,980.97 - Reuters",You won't have to stick to Bitcoin if you're d...,https://www.entrepreneur.com/article/386641,https://s.yimg.com/os/creatr-uploaded-images/2...,2021-10-11T20:28:44Z,It’s a big day for the Amazon of the decentral...
freq,6,2,1,1,1,1,1,1


### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [44]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [45]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()
# Create a list of stopwords
lemmatizer.lemmatize('Bitcoin')

# Expand the default stopwords list if necessary
sentence = "The foxes walk into the boxes."
words = word_tokenize(sentence)

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/manoloserrano/nltk_data'
    - '/Users/manoloserrano/opt/anaconda3/nltk_data'
    - '/Users/manoloserrano/opt/anaconda3/share/nltk_data'
    - '/Users/manoloserrano/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [15]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [16]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---